# 数据准备

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import warnings
import re

In [127]:
# 导入数据
path = 'E:/桌面/各专业得分.xlsx'
data1 = pd.read_excel(path)
data1.head(2)

,年份,院系名称,专业名称,落实率,月收入,专业相关度,就业满意度,离职率,教学满意度,培养目标了解度,就业适应性,母校推荐度,课程满足度,职业能力满足度
0,2022,交通工程学院,城市轨道交通工程技术,0.836364,5751.666667,0.750000,0.300000,0.571429,0.945946,0.472727,0.181818,0.554545,0.919708,0.909091
1,2022,交通工程学院,城市轨道交通运营管理,0.750000,4300.000000,0.454545,0.333333,0.619048,0.882353,0.392857,0.250000,0.500000,0.904255,0.763636


In [78]:
# 是用来看一下清洗的效果
data1.groupby(['教学满意度','教学满意程度'])['用户ID'].mean()

教学满意度  教学满意程度
0.0    不满意       2.016221e+09
       很不满意      2.016221e+09
1.0    很满意       2.016221e+09
       满意        2.016221e+09
Name: 用户ID, dtype: float64

In [128]:
data1['课程满足度'].unique()

array([0.91970803, 0.90425532, 0.84      , 0.91699605, 0.96812749,
       0.83695652, 0.75396825, 0.93478261, 0.86086957, 0.89276139,
       0.90604027, 0.82733813, 0.76470588, 0.92226148, 0.95748988,
       0.88372093, 0.76595745, 0.82248521, 0.904     , 1.        ,
       0.96186441, 0.94213382, 0.9       , 0.95595855, 0.86111111,
       0.94610778, 0.80722892, 0.88333333, 0.96028037, 0.89247312,
       0.94193548, 0.93031359, 0.93617021, 0.92444444, 0.9378882 ,
       0.85393258, 0.91752577, 0.70689655, 0.58823529, 0.80434783,
       0.85990338, 0.86046512, 0.78014184, 0.94736842, 0.73856209,
       0.8034188 , 0.76923077, 0.69090909, 0.82781457, 0.75      ,
       0.21428571, 0.51219512, 0.84491979, 0.87012987, 0.82608696,
       0.80213904, 0.73684211, 0.69387755, 0.83333333, 0.90277778,
       0.82222222, 0.77514793, 0.90196078, 0.44444444, 0.6984127 ,
       0.73275862, 0.65957447, 0.78481013, 0.88292683, 0.90551181,
       0.86173633, 0.87341772, 0.88435374, 0.95454545, 0.78571

In [111]:
data1['就业满意度']=data1['现在从事的工作是否符合职业期待'].apply(lambda x: isinstance(x,float) and x or (x=='符合' and 1 or 0))

In [112]:
data1['教学满意度_c']=data1['教学满意度'].apply(lambda x: isinstance(x,float) and x or ('不满意' in x and 1 or 2)-1)

In [113]:
data1['培养目标了解度']=data1['是否了解专业对您在知识、能力、素养方面的培养要求'].apply(lambda x: isinstance(x,float) and x or ('不了解' in x and 1 or ('部分' in x and 1.5 or 2))-1)

In [114]:
data1['就业适应性']=data1['您是否适应目前的工作岗位'].apply(lambda x: isinstance(x,float) and x or ('不' in x and 1 or 2)-1)

KeyError: '您是否适应目前的工作岗位'

In [116]:
data1['母校推荐度']=data1['是否愿意推荐本校'].apply(lambda x: isinstance(x,float) and x or ('不确定' == x and np.nan or ('不' in x and 1 or 2)-1))

In [117]:
data = data1.groupby(['院系名称','专业名称']).agg({'毕业后是否就业（1表示就业，0表示失业）':'mean',\
                                           #落实率
                                          '毕业后的月收入（元）':'mean',\
                                          '工作与专业是否相关（1表示工作与专业相关，0表示工作与专业不相关）':'mean',\
                                           #专业相关度
                                          '就业满意度':'mean',\
                                           # 由‘现在从事的工作是否符合职业期待’清洗
                                           '是否有过离职（1表示有过离职，0表示没有离职）':'mean',\
                                           '教学满意度_c':'mean',\
                                           # 由‘教学满意度’清洗 '教学满意程度'
                                           '培养目标了解度':'mean',\
                                           # 由‘是否了解专业对您在知识、能力、素养方面的培养要求’清洗
#                                            '就业适应性':'mean',\
                                           # 由‘您是否适应目前的工作岗位’清洗
                                           '母校推荐度':'mean',\
                                           # 由‘是否愿意推荐本校’清洗
                                          })
data.head(2)

毕业后是否就业（1表示就业，0表示失业）   毕业后的月收入（元）  \
院系名称   专业名称                                            
交通工程学院 城市轨道交通工程技术               0.88000  4732.142857   
       道路桥梁工程技术                 0.90625  4975.241379   

                   工作与专业是否相关（1表示工作与专业相关，0表示工作与专业不相关）     就业满意度  \
院系名称   专业名称                                                      
交通工程学院 城市轨道交通工程技术                           0.647059  0.125000   
       道路桥梁工程技术                             0.948718  0.571429   

                   是否有过离职（1表示有过离职，0表示没有离职）   教学满意度_c   培养目标了解度     母校推荐度  
院系名称   专业名称                                                               
交通工程学院 城市轨道交通工程技术                 0.250000  0.937500  0.500000  0.733333  
       道路桥梁工程技术                   0.266667  0.933333  0.630952  0.878049

In [118]:
data.to_excel('E:/桌面/各专业得分20.xlsx')

In [119]:
data1 = pd.read_excel(path,sheet_name='Course')

# data1['zynl'] = data1['课程是否满足工作或学习要求（1表示满足，0表示不满足）'].apply(lambda x:1 if x=='满足' else 0)

data1.groupby(['院系名称','专业名称']).agg({'课程是否满足工作或学习要求（1表示满足，0表示不满足）':'mean'}).to_excel('E:/桌面/12.xlsx')

In [108]:
data1 = pd.read_excel(path,sheet_name='职业能力')

data1['zynl'] = data1['职业能力是否满足需要'].apply(lambda x:1 if x=='满足' else 0)

data1.groupby(['院系名称','专业名称']).agg({'zynl':'mean'}).to_excel('E:/桌面/12.xlsx')

# 函数定义

In [132]:
# 熵值法
def cal_weight(column,row,value):
    p= np.array([[0.0 for i in range(column)] for i in range(row)])     
    for i in range(column):
        p[:,i]=value[:,i]/np.sum(value[:,i],axis=0)
    e=-1/np.log(row)*sum(p*np.log(p+1e-10))      #计算熵值
    g=1-e     # 计算一致性程度
    w=g/sum(g)     #计算权重
    return w

# 权重计算

In [126]:
group1 = ['落实率','月收入','专业相关度','就业满意度','离职率','教学满意度','培养目标了解度','就业适应性','母校推荐度','课程满足度','职业能力满足度']

In [129]:
data = data1[group1]
data = data.fillna(data.mean())

In [130]:
max_money = max(data['月收入'])

# 归一化
data['月收入'] = data['月收入'].apply(lambda x:x/max_money)
data

,落实率,月收入,专业相关度,就业满意度,离职率,教学满意度,培养目标了解度,就业适应性,母校推荐度,课程满足度,职业能力满足度
0,0.836364,0.912963,0.750000,0.300000,0.571429,0.945946,0.472727,0.181818,0.554545,0.919708,0.909091
1,0.750000,0.682540,0.454545,0.333333,0.619048,0.882353,0.392857,0.250000,0.500000,0.904255,0.763636
2,1.000000,0.732143,0.250000,0.166667,1.000000,1.000000,0.323529,0.294118,0.441176,0.840000,0.695652
3,0.800000,0.759901,0.756757,0.729730,0.369565,0.964912,0.529412,0.411765,0.658824,0.916996,0.904459
4,0.925926,0.636457,0.734375,0.620690,0.190476,0.879310,0.524691,0.654321,0.716049,0.968127,0.924603
...,...,...,...,...,...,...,...,...,...,...,...
162,0.758621,0.660562,0.294118,0.300000,0.692308,0.777778,0.500000,0.772747,0.800000,0.800000,0.730933
163,0.848485,0.662302,0.363636,0.235294,0.684211,0.791667,0.521739,0.772747,0.842105,0.743902,0.730933
164,0.680000,0.598413,0.375000,0.375000,0.666667,1.000000,0.550000,0.772747,0.909091,0.954545,0.730933
165,0.842105,0.777778,0.266667,0.333333,0.333333,0.900000,0.416667,0.772747,0.888889,0.681818,0.730933


In [135]:
data_temp = data[:125]# 2019的用不了

In [136]:
w = cal_weight(data_temp.shape[1],data_temp.shape[0],data_temp.values)
w

array([0.02745775, 0.02917741, 0.1644388 , 0.1797248 , 0.23131938,
       0.01224116, 0.02023351, 0.18050733, 0.04343622, 0.02971486,
       0.08174876])

# 分数计算

In [142]:
data['就业总分']=data.apply(lambda x:x['落实率']*w[0]+x['月收入']*w[1]+x['专业相关度']*w[2]+x['就业满意度']*w[3]+(1-x['离职率'])*w[4]+x['教学满意度']*w[5]+x['培养目标了解度']*w[6]+x['就业适应性']*w[7]+x['母校推荐度']*w[8]+x['课程满足度']*w[9]+x['职业能力满足度']*w[10],axis=1)

In [138]:
for i in range(0,11):
    print(f"x['{group1[i]}']*w[{i}]",end='+')

x['落实率']*w[0]+x['月收入']*w[1]+x['专业相关度']*w[2]+x['就业满意度']*w[3]+x['离职率']*w[4]+x['教学满意度']*w[5]+x['培养目标了解度']*w[6]+x['就业适应性']*w[7]+x['母校推荐度']*w[8]+x['课程满足度']*w[9]+x['职业能力满足度']*w[10]+

In [143]:
data.to_excel('E:/桌面/各专业得分1.xlsx')